In [1]:
from google.auth import default
credentials, project = default()

In [2]:
%load_ext autoreload
%autoreload 2
import os
import sys
from pathlib import Path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from dotenv import load_dotenv
load_dotenv()
from tqdm import tqdm

from src.utils import (load_json, get_first_n_items, save_json, save_text_file, load_text_file)  # noqa: E402
from src.story import upload_story_image, prepare_dialogue_with_wiktionary
from src.translation import review_story_dialogue_translations
from src.gcs_storage import check_blob_exists, read_from_gcs, upload_to_gcs, get_story_translated_dialogue_path, get_story_dialogue_path
from src.config_loader import config
print(config.TARGET_LANGUAGE_NAME)
COLLECTION = "WarmUp150"

Found existing FFmpeg paths:
  C:\Users\andyl\AppData\Local\Microsoft\WinGet\Packages\Gyan.FFmpeg_Microsoft.Winget.Source_8wekyb3d8bbwe\ffmpeg-7.1.1-full_build\bin
French


In [3]:
config._load_config()
language_name = config.TARGET_LANGUAGE_NAME.lower()
print(language_name)
config.get_voice_models(enum_type="stories")

french


(VoiceInfo(name='en-GB-Chirp3-HD-Algenib', provider=<VoiceProvider.GOOGLE: 'google'>, voice_id='en-GB-Chirp3-HD-Algenib', language_code='en-GB'),
 VoiceInfo(name='fr-FR-Chirp3-HD-Zephyr', provider=<VoiceProvider.GOOGLE: 'google'>, voice_id='fr-FR-Chirp3-HD-Zephyr', language_code='fr-FR'),
 VoiceInfo(name='fr-FR-Chirp3-HD-Puck', provider=<VoiceProvider.GOOGLE: 'google'>, voice_id='fr-FR-Chirp3-HD-Puck', language_code='fr-FR'))

In [4]:
from src.gcs_storage import get_stories_from_collection

all_stories = get_stories_from_collection(collection=COLLECTION)

In [5]:
all_stories

['story_mysterious_wedding_gift',
 'story_airport_wedding_chaos',
 'story_birthday_train_adventure',
 'story_underwater_community_centre',
 'story_concert_plans_unravel',
 'story_sunset_adventure_documentary',
 'story_midnight_bakery',
 'story_the_wedding_planner']

## Loop all stories (text)

Translate and re-upload

In [6]:
# loop stories and translate
from src.gcs_storage import check_blob_exists, get_story_dialogue_path, get_story_translated_dialogue_path
from src.dialogue_generation import translate_and_upload_dialogue
from src.translation import review_story_dialogue_translations

for story_name in all_stories:
    # get the dialogue
    story_file_path = get_story_dialogue_path(story_name, collection=COLLECTION)
    translated_file_path = get_story_translated_dialogue_path(story_name, collection=COLLECTION)
    if check_blob_exists(config.GCS_PRIVATE_BUCKET, translated_file_path):
        print(f"{story_name} already translated")
        #continue
    story_dialogue = read_from_gcs(config.GCS_PRIVATE_BUCKET, story_file_path)
    translate_and_upload_dialogue(story_dialogue, story_name, collection=COLLECTION)

adding translations:   0%|          | 0/3 [00:00<?, ?it/s]

Beginning translation for introduction


adding translations:  33%|███▎      | 1/3 [00:01<00:03,  1.90s/it]

Translated dialogue
Beginning translation for development


adding translations:  67%|██████▋   | 2/3 [00:03<00:01,  1.82s/it]

...

adding translations:  67%|██████▋   | 2/3 [00:03<00:01,  1.93s/it]

Translated dialogue
Beginning translation for resolution


adding translations: 100%|██████████| 3/3 [00:05<00:00,  1.90s/it]

Translated dialogue


Translated dialogue uploaded to gs://audio-language-trainer-private-content/collections/WarmUp150/french/stories/story_the_wedding_planner/translated_dialogue.json


## Review translations with LLM

In [7]:
for story_name in all_stories:
    print(f"processing {story_name}")
    # get the dialogue
    translated_file_path = get_story_translated_dialogue_path(story_name, collection=COLLECTION)
    translated_dialogue = read_from_gcs(config.GCS_PRIVATE_BUCKET, translated_file_path)
    reviewed_dialogue = review_story_dialogue_translations(translated_dialogue)
    upload_to_gcs(obj=reviewed_dialogue, bucket_name=config.GCS_PRIVATE_BUCKET, file_name=translated_file_path)

processing story_mysterious_wedding_gift
processing story_airport_wedding_chaos
processing story_birthday_train_adventure
processing story_underwater_community_centre
processing story_concert_plans_unravel
processing story_sunset_adventure_documentary
processing story_midnight_bakery
processing story_the_wedding_planner


## Add wiktionary links to each story utterance

In [ ]:
for story_name in all_stories:
    print(f"processing {story_name}")
    # get the dialogue
    translated_file_path = get_story_translated_dialogue_path(story_name, collection=COLLECTION)
    if not check_blob_exists(config.GCS_PRIVATE_BUCKET, translated_file_path):
        print(f"{story_name} not yet translated")
        continue
    translated_dialogue = read_from_gcs(config.GCS_PRIVATE_BUCKET, translated_file_path)
    translated_dialogue_with_links = prepare_dialogue_with_wiktionary(translated_dialogue)
    # now re-upload it with embedded witkionary_links
    uploaded = upload_to_gcs(obj=translated_dialogue_with_links, bucket_name=config.GCS_PRIVATE_BUCKET, file_name=translated_file_path)
    print(f"uploaded {story_name} : {uploaded}")

processing story_mysterious_wedding_gift
Got word link cache of size 3271 from GCS


Getting dialogue links for story_parts: 100%|██████████| 3/3 [00:08<00:00,  2.92s/it]


Generate audio and upload

### Generate audio and upload

### Loop through stories to generate audio

In [1]:
all_stories

NameError: name 'all_stories' is not defined

In [13]:
from src.audio_generation import generate_dialogue_audio_and_upload
for story_name in all_stories:
    # get the dialogue
    translated_file_path = get_story_translated_dialogue_path(story_name, collection=COLLECTION)

    translated_dialogue = read_from_gcs(config.GCS_PRIVATE_BUCKET, translated_file_path)
    generate_dialogue_audio_and_upload(translated_dialogue, story_name, collection=COLLECTION, overwrite=True)

Processing story parts:   0%|          | 0/3 [00:00<?, ?it/s]

Generated and uploaded: gs://audio-language-trainer-private-content/collections/LM1000/swedish/stories/story_sunset_wedding_blues/audio/introduction/part_0_alex.mp3


Generated and uploaded: gs://audio-language-trainer-private-content/collections/LM1000/swedish/stories/story_sunset_wedding_blues/audio/introduction/part_1_sam.mp3


Generated and uploaded: gs://audio-language-trainer-private-content/collections/LM1000/swedish/stories/story_sunset_wedding_blues/audio/introduction/part_2_alex.mp3


Generated and uploaded: gs://audio-language-trainer-private-content/collections/LM1000/swedish/stories/story_sunset_wedding_blues/audio/introduction/part_3_sam.mp3


...

Generated and uploaded: gs://audio-language-trainer-private-content/collections/LM1000/swedish/stories/story_job_interview_gone_wrong/audio/development/part_7_sam.mp3


Generated and uploaded: gs://audio-language-trainer-private-content/collections/LM1000/swedish/stories/story_job_interview_gone_wrong/audio/resolution/part_0_alex.mp3


Generated and uploaded: gs://audio-language-trainer-private-content/collections/LM1000/swedish/stories/story_job_interview_gone_wrong/audio/resolution/part_1_sam.mp3


Processing story parts:  67%|██████▋   | 2/3 [02:28<01:14, 74.43s/it]


Exception: ElevenLabs speech synthesis error: API request failed with status code 401: {"detail":{"status":"quota_exceeded","message":"This request exceeds your quota of 32184. You have 11 credits remaining, while 81 credits are required for this request."}}

### Fast Audio 
1 file for each story part

In [ ]:
from src.audio_generation import generate_and_upload_fast_audio

for story_name in tqdm(all_stories):
    # uploads 1 fast mp3 file for each story part. 
    generate_and_upload_fast_audio(story_name, collection=COLLECTION, overwrite=True)

## Upload challenges

In [ ]:
from src.chat import get_html_challenge_inputs, create_html_challenges
from src.gcs_storage import upload_to_gcs, get_story_challenges_path, get_story_translated_challenges_path, read_from_gcs

for story_name in all_stories:
    challenge_file_path = get_story_challenges_path(story_name, collection=COLLECTION)
    scenario_dicts = read_from_gcs(bucket_name=config.GCS_PRIVATE_BUCKET, file_path=challenge_file_path)
    challenges = get_html_challenge_inputs(scenario_dicts)
    chat_webpage_file = create_html_challenges(challenges, story_name=story_name, collection=COLLECTION) # this creates and uploades
